# GLM-MDA diffusion coefficient calculator

or *Globule-Linker-Model, Minimum-Dissipation-Approximation diffusion coefficient calculator*

<img src="https://raw.githubusercontent.com/radostw/glm-mda-diffusion/main/sequence_and_shape_banner.png">

Minimum dissipation approximation is a fast algorithm for predicting the diffusive properties of intrinsically disordered proteins.

# License

This software is licensed under GPLv3 License

Copyright (C) Radost Waszkiewicz (2023).

# How to cite

*Hydrodynamic Radii of Intrinsically Disordered Proteins: Fast Prediction by Minimum Dissipation Approximation and Experimental Validation.*
R. Waszkiewicz, A. Michaś, M. K. Białobrzewski, B. P. Klepka, M. K. Cieplak-Rotowska, Z. Staszałek, B. Cichocki, M. Lisicki, P. Szymczak, and A. Niedźwiecka; J. Phys. Chem. Lett. 15, 5024−5033  (2024)

```bibtex
@article{Waszkiewicz_2024,
	title        = {Hydrodynamic Radii of Intrinsically Disordered Proteins: Fast Prediction by Minimum Dissipation Approximation and Experimental Validation},
	author       = {Waszkiewicz, Radost and Michas, Agnieszka and Bia{\l}obrzewski, Micha{\l} K and Klepka, Barbara P and Cieplak-Rotowska, Maja K and Stasza{\l}ek, Zuzanna and Cichocki, Bogdan and Lisicki, Maciej and Szymczak, Piotr and Niedzwiecka, Anna},
	year         = 2024,
	journal      = {The Journal of Physical Chemistry Letters},
	publisher    = {ACS Publications},
	volume       = 15,
	number       = 19,
	pages        = {5024--5033}
}
```

In [ ]:
#@title install dependencies {display-mode: "form"}
!pip install glm_mda_diffusion
!pip install py3Dmol

from IPython.display import HTML, display

def display_table(data):
    html = '<table>'
    for row in data:
        html += "<tr>"
        for field in row:
            html += f'<td style="font-size: 1.2rem;">{field}</td>'
        html += "</tr>"
    html += "</table>"
    #print(html)
    display(HTML(html))

In [ ]:
#@title Input annotated protein sequence, then hit `Runtime` -> `Run all` { display-mode: "form" }

#@markdown # General parameters
sequence = '[GLPLPLKNENAIVDGDGTSVVTTKEDASTIFERDPNPANQVSAMVTGVILDENGD]PGESDESVENVDNDGEGGDKDDDKNGEDNDLDNKEHEEEKGDDDRGDDEEEDDAEGDNDSNDNEGDDDDDDDSGDDDDVDESGADEDDDDDSGD' #@param {type:"string"}
#@markdown - Use `[]` domain boundaries in the protein. For example `[GLP]LPLKNE[NAIVDGDGTSVVTGD]` has a small domain then 6 aa linker then a bigger domain.
temperature = 293.15 #@param {type:"number"}
#@markdown - Units: Kelvin
viscosity = 1.0016 #@param {type:"number"}
#@markdown - Units: mPa⋅s

#@markdown ---
ensemble_size = 50 #@param {type:"integer"}
bootstrap_rounds = 10 #@param {type:"integer"}

#@markdown ---

#@markdown # Advanced parameters
steric_radius = 1.9025 #@param {type: "number"}
#@markdown half of Ca-Ca distance, Angstroms
hydrodynamic_radius = 4.2 #@param {type: "number"}
#@markdown hydrodynamic radius of aminoacids in linkers, Angstroms
effective_density = 0.52 #@param {type: "number"}
#@markdown effective density of cores of domains, Dalton / Angstrom^3
hydration_thickness=3.0 #@param {type: "number"}
#@markdown hydration thickness on domains, Angstroms

import glm_mda_diffusion
result = glm_mda_diffusion.protein_hydrodynamic_radius(
    sequence=sequence,
    steric_radius=steric_radius,  # Ang
    hydrodynamic_radius=hydrodynamic_radius,  # Ang
    effective_density=effective_density,  # Da / Ang^3
    hydration_thickness=hydration_thickness,  # Ang
    ensemble_size=ensemble_size,
    bootstrap_rounds=bootstrap_rounds,
)

def radius_with_uncertanity(result):
  return f"{result['protein_rh']:10.2f} ± {result['protein_rh_sigma']:10.2f}"

def diffusion_with_uncertanity(result):
  rh = result['protein_rh']
  rh_sigma = result['protein_rh_sigma']
  kb = 1.380649
  diffusion = 100 * (kb * temperature) / (6 * 3.1415 * viscosity * rh)
  diffusion_sigma = diffusion * (rh_sigma / rh)
  return f"{diffusion:10.2f} ± {diffusion_sigma:10.2f}"


display_table([
    ['Hydrodynamic radius',radius_with_uncertanity(result),"Å"],
    ['Diffusion coefficient',diffusion_with_uncertanity(result),"µm² / s"],
    ])

In [ ]:
#@title Precompute shapes to visualise { display-mode: "form" }

import plotly.graph_objects as go
import numpy as np

bdc = glm_mda_diffusion.glm_mda_diffusion._bead_description_compact(
        annotated_sequence=sequence,
        steric_radius=steric_radius,
        hydrodynamic_radius=hydrodynamic_radius,
        effective_density=effective_density,
        hydration_thickness=hydration_thickness,
        aminoacid_masses=glm_mda_diffusion.glm_mda_diffusion.default_aminoacid_masses,
    )

bead_steric_radii = glm_mda_diffusion.glm_mda_diffusion._bead_steric_radii(bdc)
bead_hydrodynamic_radii = glm_mda_diffusion.glm_mda_diffusion._bead_hydrodynamic_radii(bdc)
bead_colors = np.where(np.array(bead_steric_radii) == steric_radius, "#4e79a7", "#f28e2b")

conformation_ensemble = []

ensemble_iterable = range(ensemble_size)

for i in ensemble_iterable:
    conformation_ensemble.append(
        glm_mda_diffusion.glm_mda_diffusion.sarw_spheres.generateChain(np.array(bead_steric_radii))
    )

import py3Dmol

def plot_spheres(centers, radii, colors):

    view = py3Dmol.view()

    for center, radius, color in zip(centers, radii, colors):
        view.addSphere({
            "center": {"x": center[0], "y": center[1], "z": center[2]},
            "radius": radius,
            "color": color,
            "opacity": 1.0
        })

    # view.setBackgroundColor("white")  # Optional: Change background color
    view.zoomTo()  # Automatically zoom to include all objects
    return view.show()

In [ ]:
#@title Display generated conformations { display-mode: "form" , run: "auto" }
conformation = 1 # @param {type:"slider", min:0, max:20, step:1}
plot_spheres(
    conformation_ensemble[min(conformation,ensemble_size-1)],
    bead_hydrodynamic_radii,
    bead_colors,
    )